In [496]:
import pandas as pd
import os
import csv
import psycopg2

# Configuración necesaria para que decouple detecte .env (donde están guardadas las configuraciones)
from decouple import config
from decouple import AutoConfig

config = AutoConfig(search_path=os.getcwd())

In [497]:
# Acá simplemente armamos una lista con todas las direcciones enteras de nuestros archivos csv
# Decidí armarlo en una función por si a la hora de utilizarlo se quisiera cambiar la dirección raíz
# Como predeterminado deje datasets que suele ser una dirección raíz común

def directoryLister(rootdir = 'datasets/'):

    filesDirs = []

    for rootdir, dirs, files in os.walk(rootdir):
        
        for file in files:
            
            filesDirs.append(os.path.join(rootdir, file))
        
    # print(filesDirs)
    return filesDirs

In [498]:
# Acá armamos una lista con todos los dataframes provenientes de los csv's

def dfListCreator(fileDirs):
    dataFrames = []
    for j in filesDirs:
        with open(j, 'r') as f:
            #d_reader = csv.DictReader(f)

            #get fieldnames from DictReader object and store in list
            #headers = d_reader.fieldnames
            
            dataFrames.append(pd.read_csv(j))
        f.close()
    return dataFrames
    # I had to use the column names, because of the bug that makes some rows one column larger


In [499]:
def column_checker(df,columns):
    for j in columns:
        if j not in df.columns:
            print('{} no está entre las columnas'.format(j))
            return False

In [500]:
# La función column_checker está diseñada para evitar que un dataset malo corte el proceso y permitir así que los datasets buenos sean procesados
# No la uso en este caso, por que no queda claro que espera Alkemy , y como solo tengo 3 datasets , terminaria perdiendo mucha información
# Para evitar esto hice a mano los casos necesarios. Pero en un caso real, probablemente descartaría los datasets rotos, especialmente si fueran muchos
# Dependiendo siempre de si puedo o no prescindir de la información que estos brindan
# De todas maneras desde .env se puede elegir usar el checker o no.



def columnSelector(dfList,columns, columns_alt = '[]'):
    for j in range(0,len(dfList)):
        dfList[j].columns = dfList[j].columns.str.lower()
        dfList[j].columns = dfList[j].columns.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

        if CHECKER == True:
            if column_checker(dfList[j],columns):
                dfList[j] = dfList[j][columns]

        else:
            if 'domicilio' in dfList[j].columns:
                dfList[j] = dfList[j][columns]

                dfList[j].rename(columns = {'idprovincia':'id_provincia',
                                            'iddepartamento': 'id_departamento',
                                            'categoria':'categoría',
                                            'cp':'código postal',
                                            'telefono':'número de teléfono',}, inplace = True) 

            elif 'direccion' in dfList[j].columns:
                dfList[j] = dfList[j][columns_alt]

                dfList[j].rename(columns = {'idprovincia':'id_provincia',
                                            'iddepartamento': 'id_departamento',
                                            'categoria':'categoría',
                                            'cp':'código postal',
                                            'telefono':'número de teléfono',
                                            'direccion':'domicilio'}, inplace = True)

    return dfList




In [501]:
def dataPipeline():
    
    DATASETS_ROOT_DIR = config('DATASETS_DIR')

    # Obtiene direcciones de mis archivos csv en base a mi rootdir
    fileDirs = directoryLister(DATASETS_ROOT_DIR)
    
    # Crea una lista con los dataframes en base a los csv
    dfList = dfListCreator(fileDirs)

    # Procesa cada dataframe para seleccionar ciertas columnas establecidas en la configuracion .env
    columns = ['cod_loc','idprovincia','iddepartamento','categoria','provincia','localidad','nombre','domicilio','cp','telefono','mail','web']
    columns_alt = ['cod_loc','idprovincia','iddepartamento','categoria','provincia','localidad','nombre','direccion','cp','telefono','mail','web']
    CHECKER = config('CHECKER')

    dfList = columnSelector(dfList, columns, columns_alt)

    print(dfList[0].head())

    

In [502]:
def procesarCines(df):
    # Descarta las columnas innecesarias, despues corrije la columna espacio_INCAA que tiene muchos espacios vacíos
    # Finalmente transforma el tipo a enteros y luego cuenta cada apricion de cada columna , según la provincia
    df = df[['Provincia','Butacas','Pantallas', 'espacio_INCAA']]
    
    df.loc[df['espacio_INCAA'] == 'SI' , ['espacio_INCAA']] = 1
    df.loc[df['espacio_INCAA'] == 'si' , ['espacio_INCAA']] = 1
    df.loc[df['espacio_INCAA'] != 1 , ['espacio_INCAA']] = 0
    
    df = df.dropna()
    
    df['espacio_INCAA'] = df['espacio_INCAA'].astype(int)
    df['Butacas'] = df['Butacas'].astype(int)
    df['Pantallas'] = df['Pantallas'].astype(int)

    
    a = df.groupby(['Provincia']).sum()
    #print(df)
    #a = df.groupby(["Provincia"]).count()['']
    print(a)


In [503]:
cinesDir = directoryLister('datasets/museos')
cinesDf = dfListCreator(cinesDir)
museosDf = procesarCines(cinesDf[-1]) # la última fecha
museosDf



                                 Butacas  Pantallas  espacio_INCAA
Provincia                                                         
Buenos Aires                       92722        357             19
Catamarca                           3200         12              1
Chaco                               2469         14              1
Chubut                              2682         10              4
Ciudad Autónoma de Buenos Aires    31386        153              3
Corrientes                          3370         17              1
Córdoba                            20799        105              2
Entre Ríos                          4086         17              2
Formosa                             1184          4              1
Jujuy                               2277          5              2
La Pampa                            2071          6              2
La Rioja                            1477         10              1
Mendoza                            11116         47           

In [504]:
conn = psycopg2.connect("dbname=mydb user=postgres password=postgres")

OperationalError: could not connect to server: No such file or directory
	Is the server running locally and accepting
	connections on Unix domain socket "/tmp/.s.PGSQL.5432"?
